## Straturi Noi

In continuare o sa utilizam o parte din straturile prezentate in curs.

Staturi noi:

Layer Convolutional:
* [torch.nn.Conv2d](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html)(in_channels, out_channels, kernel_size, stride=1, padding=0)

Layere Pooling:
* [torch.nn.MaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)(kernel_size, stride=None, padding=0)
*  [torch.nn.AveragePool2d](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html)(kernel_size, stride=None, padding=0)

Layere Adaptive Pool, intalnit adesea si ca Global Pool:
* [torch.nn.AdaptiveAvgPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveAvgPool2d.html)(output_size)
* [torch.nn.AdaptiveMaxPool2d](https://pytorch.org/docs/stable/generated/torch.nn.AdaptiveMaxPool2d.html)(output_size)

Layer de liniarizare:

* [torch.nn.Flatten()](https://pytorch.org/docs/stable/generated/torch.flatten.html)



Four **hyperparameters** control the size of the output volume:
* **Depth**: number of filters, as each filter _looks_ at different areas of the input:
* **Stride**: the step taken when _sliding_ the filter. (Usually 1 or 2, 3 - uncommon).
* **Zero-Padding**: size of the number of 0s that surround the border of the input volume. Example: If you want to the same width and height for input and output.
* **Dilation**: Distance between elements of the convolutional kernel.



**Why Pooling Layer?**

1. Modifica volumul de input (input volume) in reprezentari _mai mici_ si mai usor de _manevrat_.
2. Opereaza independent pe fiecare Activation Map.

<img src="https://computersciencewiki.org/images/9/9e/MaxpoolSample.png" width="425" height="300"/> <img src="https://miro.medium.com/v2/resize:fit:517/0*lIcR0gOMK0xzTr6_.png" width="425" height="300"/> 






**Why Adaptive Pooling Layer?**

1. Folosite de regula in etapele finale de constructie a unei arhitecturi de tipul _ConvNet_ pentru a inlocui fully-connected layers.
2. Incearca sa previna *overfitting phenomenon* fortand feature maps sa retina informatia **globala** care este relevanta pentru task-ul acestei _ConvNet_ (clasificare, identifcare etc.)

<img src="https://www.researchgate.net/publication/338079465/figure/fig4/AS:905983672987648@1593014748012/The-difference-of-max-pooling-and-global-max-pooling.ppm" width="725" height="300"/>


<img src="https://drive.google.com/uc?id=11l7Xsh-iQmASvXTkgH2MgtA01XCW6CAC">


[Visualise them Here](https://github.com/vdumoulin/conv_arithmetic).

In [4]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(2,2))
print("Conv1 result shape",layer(dummy_input_tensor).shape)

layer = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(13,13), stride=(2,2))
print("Conv2 result shape",layer(dummy_input_tensor).shape)

layer = nn.MaxPool2d(kernel_size=(3,3)) # Stride este inferat din kernel size, ca fiind egal cu kernel size ca sa nu repete elementele luate
print("Pool result shape",layer(dummy_input_tensor).shape)

# Utilizate pentru a reduce dimensiunea la una prestabilita, util cand marimea input ului este variabil
layer = nn.AdaptiveAvgPool2d(output_size=(5,5))
print("Global Pool result shape",layer(dummy_input_tensor).shape)

layer = nn.Flatten()
print("Flaten result shape",layer(dummy_input_tensor).shape)

Conv1 result shape torch.Size([1, 10, 49, 49])
Conv2 result shape torch.Size([1, 10, 44, 44])
Pool result shape torch.Size([1, 3, 33, 33])
Global Pool result shape torch.Size([1, 3, 5, 5])
Flaten result shape torch.Size([1, 30000])


###Cerinte

**(1p)** Utilizati o serie de Conv2D/Pool2D pentru a ajunge la urmatoarele marimi plecand de la input 3x100x100:
*   [1, 10, 25, 25] # Stride & Padding
*   [1, 10, 32, 32]
*  [1, 3, 2, 2]



In [1]:
import numpy as np
import torch.nn as nn
import torch

dummy_input_tensor = torch.rand((1,3,100,100))  # Input random de marime 100x100 cu 3 canale


# [1, 10, 25, 25]
layer1 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(6, 6), padding = 25)
print(layer1(dummy_input_tensor).shape)

# [1, 10, 32, 32]
layer2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=(3,3), stride=(4, 4), padding = 14)
print(layer2(dummy_input_tensor).shape)

# [1, 3, 2, 2]
layer3 = nn.Conv2d(in_channels=3, out_channels=3, kernel_size=(25, 25), stride=(50, 50))
print(layer3(dummy_input_tensor).shape)





torch.Size([1, 10, 25, 25])
torch.Size([1, 10, 32, 32])
torch.Size([1, 3, 2, 2])


## Instantierea seturilor de date

In [2]:
import torchvision

cifar_train = torchvision.datasets.CIFAR10("./data", download=True)
cifar_test = torchvision.datasets.CIFAR10("./data", train=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data


## Crearea Dataloader-ului

### Cerinte
 * **(2p)** Implementati functia de preprocesare a datelor, __collate_fn(examples)__.


Atentie! Spre deosebire de intrarea pentru retelele fully-connected, pentru retelele convolutionale intrearea nu trebuie liniarizata, ci doar normalizata.

#### Hint

  * Amintiti-va folosirea functiei __normalize__ din torchvision.transforms.functional din laboratorul trecut.
  * Modificati functia *collate_fn* din laboratorul trecut, pentru a normaliza datele in intervalul [-1, 1]

In [3]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def collate_fn(examples):
  ### Completati codul pentru cerinta aici
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0])
    # In linia de mai jos imaginea este normalizata astfel incat sa aiba toate valorile in 
    # [-1, 1] in loc de [0, 255]
    normalized_tensor_image = normalize(tensor_image, [0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    normalized_tensor_image = normalized_tensor_image.unsqueeze(0)

    processed_images.append(normalized_tensor_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)

  return torch_images, torch_labels

train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

## Crearea unei retele neurale convolutionale

### Cerinte
 * **(1p)** Creati o clasa care mosteneste clasa nn.Module. Ea va reprezenta o retea neurala convolutionala pentru clasificare ale celor 10 clase din datasetul CIFAR10.
    * Reteaua trebuie sa aiba 2 straturi convolutionale care sa reduca dimensiunea spatiala a imaginii de 2 ori (0.25 p).
    * Liniarizati iesirea din cel de-al doilea strat convolutional (0.25 p).
    * Adaugat stratul final de tipul `fully-connected` (0.25 p).
    * Folositi o functie de activare la alegere (Exemplu [RELU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)) (0.25 p).

#### Hint

Pentru a liniariza iesirea din cel de-al doilea feature map puteti adopta mai multe strategii:
  * Liniarizare prin schimbarea shape-ului la [batch_size, -1]
  * Global Max Pooling si apoi liniarizare la [batch_size, -1]
  * Average Max Pooling si apoi liniarizare la [batch_size, -1]

In [4]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    ## din 32x32x3 -> layer1 -> activation1 -> 3x16x16
    self.layer1 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = (3, 3), stride = (2, 2), padding=1)
    ## din 16x16x3 -> layer2 -> activation2 -> 3x8x8
    self.layer2 = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = (3, 3), stride = (2, 2), padding=1)

    self.fully_connected_layer = nn.Linear(8*8*3, 10, bias=False) 
    self.activation = nn.ReLU()
    
  def forward(self, x):
    x = self.layer1(x)
    x = self.activation(x)
    x = self.layer2(x)
    x = self.activation(x)
    x = x.reshape((-1, 8*8*3))
    x = self.fully_connected_layer(x)
    x = self.activation(x)
    return x

## Definirea obiectelor folosite in timpul antrenarii

### Cerinte **(1p)**
  * Numarul de epoci (0.25 p)
  * Retea (0.25 p)
  * Optimizator (0.25 p)
  * Alegeti functia de cost (0.25 p)

In [5]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
network = Net()

# Definiti optimizatorul
optimizer = optim.SGD(network.parameters(), lr=1e-2)
"""
Dupa definirea optimizatorului si dupa fiecare iteratie de antrenare, trebuie 
apelata functia zero_grad() pentru a seta valoare tuturor gradientilor la zero.
"""
# Completati aici codul pentru seta valoare tuturor gradientilor la zero
optimizer.zero_grad()

# Definiti functia cost pentru clasificare Cross-Entropy
# https://neptune.ai/blog/pytorch-loss-functions
loss_fn = nn.CrossEntropyLoss()

## Definirea functiei de antrenare

In [6]:
from torch.utils.data import DataLoader

In [18]:
def test_acc(net: nn.Module, test_loader: DataLoader):
  net.eval()

  total = 0
  correct = 0

  for test_images, test_labels in test_loader:
    total += len(test_images)
    out_class = torch.argmax(net(test_images))
    correct += torch.sum(out_class == test_labels)

  return correct / total * 100


def train_fn(epochs: int, train_loader: DataLoader, test_loader: DataLoader, 
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer):
  # Iteram prin numarul de epoci
  for e in range(epochs):
    net.train()

    # Iteram prin fiecare batch din dataloader
    for images, labels in train_loader:
      # Aplicam reteaua neurala pe imaginile din batch-ul curent
      out = net(images)
      # Aplicam functia cost pe iesirea retelei neurale si pe etichetele imaginilor 
      loss = loss_fn(out, labels)
      # Aplicam algoritmul de back-propagation
      loss.backward()
      # Facem pasul de optimizare, pentru a actualiza parametrii retelei
      optimizer.step()
      # Apelam functia zero_grad() pentru a uita gradientii de la iteratie curenta
      optimizer.zero_grad()
    
    print("Loss-ul la finalul epocii {} are valoarea {}".format(e, loss.item()))

    # Calculam acuratetea
    acc = test_acc(net, test_loader)
    print("Acuratetea la finalul epocii {} este {:.2f}%".format(e, acc))


## Antrenarea

### Cerinte
  * Antrenati reteaua definita mai sus (clasa Net)

In [18]:
train_fn(epochs, train_loader, test_loader, network, loss_fn, optimizer)

Loss-ul la finalul epocii 0 are valoarea 2.3024702072143555
Acuratetea la finalul epocii 1 este 9.93%
Loss-ul la finalul epocii 1 are valoarea 2.301344633102417
Acuratetea la finalul epocii 2 este 10.49%
Loss-ul la finalul epocii 2 are valoarea 2.3025829792022705
Acuratetea la finalul epocii 3 este 10.72%
Loss-ul la finalul epocii 3 are valoarea 2.2998664379119873
Acuratetea la finalul epocii 4 este 11.03%
Loss-ul la finalul epocii 4 are valoarea 2.3001582622528076
Acuratetea la finalul epocii 5 este 11.03%
Loss-ul la finalul epocii 5 are valoarea 2.2987558841705322
Acuratetea la finalul epocii 6 este 10.91%
Loss-ul la finalul epocii 6 are valoarea 2.301168918609619
Acuratetea la finalul epocii 7 este 10.91%
Loss-ul la finalul epocii 7 are valoarea 2.2974815368652344
Acuratetea la finalul epocii 8 este 10.68%
Loss-ul la finalul epocii 8 are valoarea 2.292241096496582
Acuratetea la finalul epocii 9 este 10.66%
Loss-ul la finalul epocii 9 are valoarea 2.296569585800171
Acuratetea la fina

## Reteaua LeNet

### Cerinte
  * **(3p)** Implementati reteaua LeNet dupa figura de mai jos si antrenati-o.



![alt text](https://drive.google.com/uc?id=1OVancUyIViMRMZdULFSVCvXJHQP0NGUV)

Figura arhitectura LeNet

![alt text](https://debuggercafe.com/wp-content/uploads/2019/07/Layers-in-LeNet.png)

Tabel arhitectura LeNet

_Question:_ Care este diferenta dintre `tanh` si `softmax`? De ce credeti ca peste ultimul layer (cel de output) a fost aplicata functia `softmax`?


In [8]:
import torch.nn as nn

class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    
    ### Completati codul pentru cerinta aici
    # input -> 1x1x32x32
    # input -> layer1 ->  1x6x28x28
    self.layer1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = (5, 5), stride = (1, 1))
    # 1x6x28x28 -> layer2 -> 1x6x14x14
    self.layer2 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x6x14x14 -> layer3 -> 1x16x10x10
    self.layer3 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = (5, 5), stride = (1, 1))
    # 1x16x10x10 -> layer4 -> 1x16x5x5
    self.layer4 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x16x5x5 -> layer5 -> 1x120x1x1
    self.layer5 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (5, 5), stride = (1, 1))

    self.fully_connected_layer1 = nn.Linear(120, 84, bias=False) 
    self.fully_connected_layer2 = nn.Linear(84, 10, bias=False) 

    self.activation1 = nn.Tanh()
    self.activation2 = nn.Softmax()

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    ### Completati codul pentru cerinta aici
    x = self.layer1(x)
    x = self.activation1(x)

    x = self.layer2(x)
    x = self.activation1(x)

    x = self.layer3(x)
    x = self.activation1(x)

    x = self.layer4(x)
    x = self.activation1(x)

    x = self.layer5(x)
    x = self.activation1(x)

    x = x.reshape((-1, 120))

    x = self.fully_connected_layer1(x)
    x = self.activation1(x)

    x = self.fully_connected_layer2(x)
    x = self.activation2(x)

    return x

## Optional: Reteaua AlexNet ❤️

❗Daca alegeti aceasta retea veti continua sa rezolvati exercitiile urmatoare pentru reteaua AlexNet.

Pentru a usura volumul de munca si obtine o retea AlexNet comparabila in dificultate cu LeNet, urmati acesti pasi:

✔️ Includeti functii de activare intre layere (exemplu ReLU).

✔️ Va folositi doar de prima subsectiune din schema figurii arhitecturii AlexNet (adica doar Conv1 si Conv2 blocks).

✔️ Inputul vostru se opreste la un minimum size de 8x8.

✔️ Modificati output-ul retelei sa prezica 10 clase in loc de 1000 de clase.





![alt text](https://anhreynolds.com/img/alexnet.png)

Figura arhitectura AlexNet.

![alt text](https://anhreynolds.com/img/alexnet-parameters.png)

Tabel arhitectura AlexNet


In [ ]:
import torch.nn as nn

class AlexNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    pass

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    return x

## Redefinirea obiectelor folosite in timpul antrenarii pentru reteaua LeNet

### Cerinta
 * Redefiniti obiectele pentru a antrena reteaua LeNet

In [9]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-2)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
lenet_optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

## Antrenarea retelei LeNet

In [22]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

<ipython-input-20-421d6b342faf>:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation2(x)


Loss-ul la finalul epocii 0 are valoarea 2.302060604095459
Acuratetea la finalul epocii 1 este 12.38%
Loss-ul la finalul epocii 1 are valoarea 2.3019416332244873
Acuratetea la finalul epocii 2 este 12.96%
Loss-ul la finalul epocii 2 are valoarea 2.3018908500671387
Acuratetea la finalul epocii 3 este 13.61%
Loss-ul la finalul epocii 3 are valoarea 2.301589250564575
Acuratetea la finalul epocii 4 este 14.32%
Loss-ul la finalul epocii 4 are valoarea 2.3017125129699707
Acuratetea la finalul epocii 5 este 14.78%
Loss-ul la finalul epocii 5 are valoarea 2.301441192626953
Acuratetea la finalul epocii 6 este 15.45%
Loss-ul la finalul epocii 6 are valoarea 2.301663875579834
Acuratetea la finalul epocii 7 este 15.99%
Loss-ul la finalul epocii 7 are valoarea 2.3011982440948486
Acuratetea la finalul epocii 8 este 16.20%
Loss-ul la finalul epocii 8 are valoarea 2.3015010356903076
Acuratetea la finalul epocii 9 este 16.56%
Loss-ul la finalul epocii 9 are valoarea 2.3010897636413574
Acuratetea la fin

###Augmentare retea

Reteaua de mai devreme duce lipsa de regularizare. O forma foarte puternica de regularizare este normalizarea, iar pentru acest lucru exista straturi speciale.

Astfel de straturi:

* [torch.nn.BatchNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html) (num_features)
* [torch.nn.InstanceNorm2d](https://pytorch.org/docs/stable/generated/torch.nn.InstanceNorm2d.html) (num_features)

Un alt element important il reprezinta functiile de activare, care pot influenta convergenta si puterea retelei. Cateva exemple de alte functii de activare:

* ReLU
* Sigmoid
* Tanh
* LeakyRelu
* GELU

## Cerinta

**(2p)** Experimentati cu aceste elemente in cadrul retelei LeNet definita mai devreme, pentru a obtine o acuratete mai buna. Observati viteza de convergenta si performanta retelei pentru 3 configuratii diferite.

**Punctaj:** 0.6p / configuratie.

0.6p din care:
- 0.4p modificarea retelei
- 0.1p obtinerea rezultatelor
- 0.1p afisarea acestora si explicatie.


###Bonus
**(1p)** Antrenati reteaua folosind GPU (Graphics processing unit)








  


In [22]:
import torch.nn as nn
# DEFAULT
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    
    ### Completati codul pentru cerinta aici
    # input -> 1x1x32x32
    # input -> layer1 ->  1x6x28x28
    self.layer1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = (5, 5), stride = (1, 1))
    # 1x6x28x28 -> layer2 -> 1x6x14x14
    self.layer2 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x6x14x14 -> layer3 -> 1x16x10x10
    self.layer3 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = (5, 5), stride = (1, 1))
    # 1x16x10x10 -> layer4 -> 1x16x5x5
    self.layer4 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x16x5x5 -> layer5 -> 1x120x1x1
    self.layer5 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (5, 5), stride = (1, 1))

    self.fully_connected_layer1 = nn.Linear(120, 84, bias=False) 
    self.fully_connected_layer2 = nn.Linear(84, 10, bias=False) 

    self.activation1 = nn.Tanh()
    self.activation2 = nn.Softmax()

  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    ### Completati codul pentru cerinta aici
    x = self.layer1(x)
    x = self.activation1(x)

    x = self.layer2(x)
    x = self.activation1(x)

    x = self.layer3(x)
    x = self.activation1(x)

    x = self.layer4(x)
    x = self.activation1(x)

    x = self.layer5(x)
    x = self.activation1(x)

    x = x.reshape((-1, 120))

    x = self.fully_connected_layer1(x)
    x = self.activation1(x)

    x = self.fully_connected_layer2(x)
    x = self.activation2(x)

    return x

In [23]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-2)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
lenet_optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

In [24]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

<ipython-input-22-8e9e11e1e750>:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation2(x)


Loss-ul la finalul epocii 0 are valoarea 2.302436113357544
Acuratetea la finalul epocii 0 este 11.59%
Loss-ul la finalul epocii 1 are valoarea 2.302324056625366
Acuratetea la finalul epocii 1 este 12.38%
Loss-ul la finalul epocii 2 are valoarea 2.3023316860198975
Acuratetea la finalul epocii 2 este 13.08%
Loss-ul la finalul epocii 3 are valoarea 2.3020753860473633
Acuratetea la finalul epocii 3 este 13.68%
Loss-ul la finalul epocii 4 are valoarea 2.302049160003662
Acuratetea la finalul epocii 4 este 14.25%
Loss-ul la finalul epocii 5 are valoarea 2.301835060119629
Acuratetea la finalul epocii 5 este 14.58%
Loss-ul la finalul epocii 6 are valoarea 2.3020224571228027
Acuratetea la finalul epocii 6 este 15.12%
Loss-ul la finalul epocii 7 are valoarea 2.3015427589416504
Acuratetea la finalul epocii 7 este 15.72%
Loss-ul la finalul epocii 8 are valoarea 2.3017613887786865
Acuratetea la finalul epocii 8 este 15.94%
Loss-ul la finalul epocii 9 are valoarea 2.3014843463897705
Acuratetea la fin

In [31]:
import torch.nn as nn
# DEFAULT + NORMALIZARE
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    
    ### Completati codul pentru cerinta aici
    # input -> 1x1x32x32
    # input -> layer1 ->  1x6x28x28
    self.layer1 = nn.Conv2d(in_channels = 3, out_channels = 6, kernel_size = (5, 5), stride = (1, 1))
    # 1x6x28x28 -> layer2 -> 1x6x14x14
    self.layer2 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x6x14x14 -> layer3 -> 1x16x10x10
    self.layer3 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = (5, 5), stride = (1, 1))
    # 1x16x10x10 -> layer4 -> 1x16x5x5
    self.layer4 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x16x5x5 -> layer5 -> 1x120x1x1
    self.layer5 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (5, 5), stride = (1, 1))

    self.fully_connected_layer1 = nn.Linear(120, 84, bias=False) 
    self.fully_connected_layer2 = nn.Linear(84, 10, bias=False) 

    self.activation1 = nn.Tanh()
    self.activation2 = nn.Softmax()

    self.normalization1 = nn.BatchNorm2d(num_features=6)
    self.normalization2 = nn.BatchNorm2d(num_features=16)
    self.normalization3 = nn.BatchNorm2d(num_features=120)


  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    ### Completati codul pentru cerinta aici
    x = self.layer1(x)
    x = self.normalization1(x)
    x = self.activation1(x)

    x = self.layer2(x)
    # x = self.activation1(x)

    x = self.layer3(x)
    x = self.normalization2(x)
    x = self.activation1(x)

    x = self.layer4(x)
    # x = self.activation1(x)

    x = self.layer5(x)
    x = self.normalization3(x)
    x = self.activation1(x)

    x = x.reshape((-1, 120))

    x = self.fully_connected_layer1(x)
    x = self.activation1(x)

    x = self.fully_connected_layer2(x)
    x = self.activation2(x)

    return x

In [32]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=1e-2)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
lenet_optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

In [33]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

<ipython-input-31-1b3d9746ce56>:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation2(x)


Loss-ul la finalul epocii 0 are valoarea 2.2963500022888184
Acuratetea la finalul epocii 0 este 15.13%
Loss-ul la finalul epocii 1 are valoarea 2.28709077835083
Acuratetea la finalul epocii 1 este 18.33%
Loss-ul la finalul epocii 2 are valoarea 2.276447057723999
Acuratetea la finalul epocii 2 este 20.19%
Loss-ul la finalul epocii 3 are valoarea 2.2688655853271484
Acuratetea la finalul epocii 3 este 20.17%
Loss-ul la finalul epocii 4 are valoarea 2.254565715789795
Acuratetea la finalul epocii 4 este 20.30%
Loss-ul la finalul epocii 5 are valoarea 2.2562661170959473
Acuratetea la finalul epocii 5 este 20.37%
Loss-ul la finalul epocii 6 are valoarea 2.243040084838867
Acuratetea la finalul epocii 6 este 20.71%
Loss-ul la finalul epocii 7 are valoarea 2.2277400493621826
Acuratetea la finalul epocii 7 este 20.94%
Loss-ul la finalul epocii 8 are valoarea 2.2499380111694336
Acuratetea la finalul epocii 8 este 21.52%
Loss-ul la finalul epocii 9 are valoarea 2.2439651489257812
Acuratetea la fina

In [51]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision.transforms.functional import to_tensor, normalize

def collate_fn(examples):
  ### Completati codul pentru cerinta aici
  processed_images = []
  processed_labels = []

  for example in examples:
    tensor_image = to_tensor(example[0].convert('L'))
    # In linia de mai jos imaginea este normalizata astfel incat sa aiba toate valorile in 
    # [-1, 1] in loc de [0, 255]
    normalized_tensor_image = normalize(tensor_image, [0.5], [0.5])
    normalized_tensor_image = normalized_tensor_image.unsqueeze(0)

    processed_images.append(normalized_tensor_image)
    
    label = np.array(example[1])
    tensor_label = torch.tensor(label)
    tensor_label = tensor_label.unsqueeze(0)
    processed_labels.append(tensor_label)

  torch_images = torch.cat(processed_images, dim=0)
  torch_labels = torch.cat(processed_labels, dim=0)

  return torch_images, torch_labels

train_loader = DataLoader(cifar_train, batch_size=500, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(cifar_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

In [52]:
import torch.nn as nn
# DEFAULT + NORMALIZARE + ALTE FUNCTII ACTIVARE
class LeNet(nn.Module):
  def __init__(self):
    super().__init__()
    """
    Punctaj: 2.5p
    """
    
    ### Completati codul pentru cerinta aici
    # input -> 1x1x32x32
    # input -> layer1 ->  1x6x28x28
    self.layer1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = (5, 5), stride = (1, 1))
    # 1x6x28x28 -> layer2 -> 1x6x14x14
    self.layer2 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x6x14x14 -> layer3 -> 1x16x10x10
    self.layer3 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = (5, 5), stride = (1, 1))
    # 1x16x10x10 -> layer4 -> 1x16x5x5
    self.layer4 = nn.AvgPool2d(kernel_size = (2, 2), stride = (2, 2))
    # 1x16x5x5 -> layer5 -> 1x120x1x1
    self.layer5 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (5, 5), stride = (1, 1))

    self.fully_connected_layer1 = nn.Linear(120, 84, bias=False) 
    self.fully_connected_layer2 = nn.Linear(84, 10, bias=False) 

    self.activation1 = nn.SELU()
    self.activation2 = nn.Softmax()

    self.normalization1 = nn.BatchNorm2d(num_features=6)
    self.normalization2 = nn.BatchNorm2d(num_features=16)
    self.normalization3 = nn.BatchNorm2d(num_features=120)


  def forward(self,x):
    """
    Punctaj: 0.5p
    """
    ### Completati codul pentru cerinta aici
    x = self.layer1(x)
    x = self.normalization1(x)
    x = self.activation1(x)

    x = self.layer2(x)
    # x = self.activation1(x)

    x = self.layer3(x)
    x = self.normalization2(x)
    x = self.activation1(x)

    x = self.layer4(x)
    # x = self.activation1(x)

    x = self.layer5(x)
    x = self.normalization3(x)
    x = self.activation1(x)

    x = x.reshape((-1, 120))

    x = self.fully_connected_layer1(x)
    x = self.activation1(x)

    x = self.fully_connected_layer2(x)
    x = self.activation2(x)

    return x

In [60]:
import torch.optim as optim

# Definiti numarul de epoci
epochs = 10

# Definiti reteaua
lenet = LeNet()

# Definiti optimizatorul
lenet_optimizer = optim.SGD(lenet.parameters(), lr=0.1)
# Dupa definirea optimizatorului si dupa fiecare iteratie trebuie apelata functia zero_grad().
# Aceasta face toti gradientii zero.
# Completati codul pentru a face gradientii zero aici
lenet_optimizer.zero_grad()


# Definiti functia cost pentru clasificare Cross-Entropy
loss_fn = nn.CrossEntropyLoss()

In [45]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

<ipython-input-43-75e2d6dc5693>:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation2(x)


Loss-ul la finalul epocii 0 are valoarea 2.278597354888916
Acuratetea la finalul epocii 0 este 22.89%
Loss-ul la finalul epocii 1 are valoarea 2.252464771270752
Acuratetea la finalul epocii 1 este 24.54%
Loss-ul la finalul epocii 2 are valoarea 2.2264788150787354
Acuratetea la finalul epocii 2 este 26.19%
Loss-ul la finalul epocii 3 are valoarea 2.221421003341675
Acuratetea la finalul epocii 3 este 27.64%
Loss-ul la finalul epocii 4 are valoarea 2.1977717876434326
Acuratetea la finalul epocii 4 este 29.76%
Loss-ul la finalul epocii 5 are valoarea 2.1969754695892334
Acuratetea la finalul epocii 5 este 31.43%
Loss-ul la finalul epocii 6 are valoarea 2.1907174587249756
Acuratetea la finalul epocii 6 este 32.48%
Loss-ul la finalul epocii 7 are valoarea 2.169053554534912
Acuratetea la finalul epocii 7 este 33.46%
Loss-ul la finalul epocii 8 are valoarea 2.1550779342651367
Acuratetea la finalul epocii 8 este 34.21%
Loss-ul la finalul epocii 9 are valoarea 2.1410598754882812
Acuratetea la fin

In [61]:
train_fn(epochs, train_loader, test_loader, lenet, loss_fn, lenet_optimizer)

<ipython-input-52-979baae17cd3>:63: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.activation2(x)


Loss-ul la finalul epocii 0 are valoarea 2.216125011444092
Acuratetea la finalul epocii 0 este 26.84%
Loss-ul la finalul epocii 1 are valoarea 2.1785318851470947
Acuratetea la finalul epocii 1 este 30.31%
Loss-ul la finalul epocii 2 are valoarea 2.1264913082122803
Acuratetea la finalul epocii 2 este 31.97%
Loss-ul la finalul epocii 3 are valoarea 2.113781213760376
Acuratetea la finalul epocii 3 este 34.31%
Loss-ul la finalul epocii 4 are valoarea 2.097304105758667
Acuratetea la finalul epocii 4 este 32.34%
Loss-ul la finalul epocii 5 are valoarea 2.0740599632263184
Acuratetea la finalul epocii 5 este 37.17%
Loss-ul la finalul epocii 6 are valoarea 2.064319133758545
Acuratetea la finalul epocii 6 este 37.01%
Loss-ul la finalul epocii 7 are valoarea 2.0350029468536377
Acuratetea la finalul epocii 7 este 41.25%
Loss-ul la finalul epocii 8 are valoarea 2.037080764770508
Acuratetea la finalul epocii 8 este 39.82%
Loss-ul la finalul epocii 9 are valoarea 2.0257980823516846
Acuratetea la fina